In [1]:
#
# //TODO:
# * Add input verification from SUG
#

In [3]:
#
# 1. A bunch of imports
#
from datetime import timedelta
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import json
import logging
import numpy as np
import pandas as pd
import re
import requests

In [4]:
#
# 2. Useful constants
#
sug_signup_url = 'https://www.signupgenius.com/go/8050b45a4a92ba6fe3-rosa/135464402#/'
sug_data_url   = 'https://www.signupgenius.com/SUGboxAPI.cfm?go=s.getSignupInfo'
sug_post_data  = '{"forSignUpView":true,"urlid":"8050b45a4a92ba6fe3-rosa","portalid":0}'

gcal_calendar_id = 'fe4d32da241b620ef20354fbcbc072f23811169be9fd209f35d8c679d65611ff@group.calendar.google.com'
gcal_auth_json   = 'C:/Users/kylew/scripts/sfusd/RPES/rpes-390404-762f98d89111.json'

logging_name = 'RPESChickens'
logging_file = 'C:/Users/kylew/scripts/sfusd/RPES/rpes-chickens.log'

In [5]:
#
# 3. Structure a Google event
#
def buildEvent ( row ):
    return {
        'summary':     f"Chickens: {row['caregiver']}",
        'start':       {
            'date':     row['date'].strftime( '%Y-%m-%d' ), # .to_pydatetime()
            'timeZone': 'America/Los_Angeles'
        },
        'end':         {
            'date':     ( row['date'] + timedelta( days = 1 ) ).strftime( '%Y-%m-%d' ), # .to_pydatetime()
            'timeZone': 'America/Los_Angeles'
        },
        'location':    'Rosa Parks Elementary School, 1501 O\'Farrell Street, San Francisco, CA, USA 94115',
        'description': f'''
{row['caregiver']}

Care instructions at https://docs.google.com/document/d/17lCqSMVdZX2srMdmZCQH4qPCH83v3dvz0Ez5EmIY8as/edit

Care signup link at https://www.signupgenius.com/go/8050b45a4a92ba6fe3-rosa/135464402
''',
        'reminders':   {
            'useDefault': False
        }
    }

In [6]:
#
# 3.5. Build logging
#
logger = logging.getLogger( logging_name )
logger.setLevel( logging.INFO )

formatter = logging.Formatter( '%(asctime)s - %(name)s - %(levelname)s - %(message)s' )

stream_console = logging.StreamHandler()
stream_file    = logging.FileHandler( logging_file )
stream_console.setLevel( logging.INFO )
stream_file.setLevel( logging.INFO )
stream_console.setFormatter( formatter )
stream_file.setFormatter( formatter )

logger.addHandler( stream_console )
logger.addHandler( stream_file )

In [7]:
#
# 4. Get the data from SUG
#
response = requests.get( sug_signup_url )
response = requests.post( sug_data_url, data = sug_post_data )

# TODO: validate response
page = json.loads( response.text )



# Load the calendar into a dataframe
dates  = []
people = []

for key, participant in page['DATA']['participants'].items():
    #print( participant )
    participant = { _key.upper(): _value for _key, _value in participant[0].items() }
    dates.append( participant['STARTTIME'] )
    people.append( f"{participant['FIRSTNAME']} {participant['LASTNAME']}" )
    
calendar = pd.DataFrame.from_dict( { 'date': pd.to_datetime( dates ), 'caregiver': people } )

C:\Users\kylew\AppData\Local\Temp\ipykernel_17072\461470586.py:22: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  calendar = pd.DataFrame.from_dict( { 'date': pd.to_datetime( dates ), 'caregiver': people } )


In [8]:
#
# 5. Auth into Google, get current Google calendar
#
with open( gcal_auth_json, 'r' ) as cred_file:
    creds = service_account.Credentials.from_service_account_info( json.load( cred_file ) )
    
service = build('calendar', 'v3', credentials=creds)



# Get the calendar info
# * Previously appended 'Z' to set to UTC (I think); new datestamps have time zone appended
# date_start = np.min( calendar['date'] ).to_pydatetime().isoformat()# + 'Z'
# date_end   = ( np.max( calendar['date'] ).to_pydatetime() + timedelta( days = 1 ) ).isoformat()# + 'Z'
date_start = np.min( calendar['date'] ).isoformat()
date_end   = ( np.max( calendar['date'] ) + timedelta( days = 1 ) ).isoformat()

events_result = service.events().list(
        calendarId   = gcal_calendar_id,
        timeMin      = date_start,
        timeMax      = date_end,
        maxResults   = 2500,
        singleEvents = True,
        orderBy      = 'startTime'
    ).execute()



events = events_result.get('items', [])

dates      = []
event_ids  = []
caregivers = []
for event in events:
    dates.append( event['start'].get( 'dateTime', event['start'].get( 'date' ) ) ) # fallback
    caregivers.append( re.sub( r'^(?:Chickens: )?', '', event['summary'] ) )
    event_ids.append( event['id'] )
    
    
    
existing_calendar = pd.DataFrame \
    .from_dict( {
        'date':           pd.to_datetime( dates, utc = True ),
        'caregiver_gcal': caregivers,
        'event_id':       event_ids
    } ) \
    .assign( date = lambda _df: _df['date'].dt.date )

In [9]:
#
# 6. Put the calendars togther, insert where necessary
#
merged_calendar = calendar \
    .assign( date = pd.to_datetime( calendar['date'], utc = True ).dt.date ) \
    .merge( existing_calendar, on = 'date', how = 'outer' )
    # .assign( date = pd.to_datetime( calendar['date'].dt.date ) ) \



# delete entries where people have removed themselves from SUG
entries_to_delete = merged_calendar[pd.isnull( merged_calendar['caregiver'] )]
if len( entries_to_delete ) > 0:
    logger.info( 'Deleting entries...' )
else:
    logger.info( 'No entries to be deleted.' )
    
for ix, row in entries_to_delete.iterrows():
    logger.info( f"* {row['date']}: {row['caregiver_gcal']}" )
    service.events().delete( calendarId = gcal_calendar_id, eventId = row['event_id'] ).execute()



# add entries where people have inserted themselves
entries_to_insert = merged_calendar[pd.isnull( merged_calendar['caregiver_gcal'] )]
if len( entries_to_insert ) > 0:
    logger.info( 'Inserting entries...' )
else:
    logger.info( 'No entries to be inserted.' )
    
for ix, row in entries_to_insert.iterrows():
    logger.info( f"* {row['date']}: {row['caregiver']}" )
    event  = buildEvent( row )
    result = service.events().insert( calendarId = gcal_calendar_id, body = event ).execute()



# update entries that have been updated on SUG
entries_to_update = merged_calendar[
        ( ~pd.isnull( merged_calendar['caregiver'] ) )
        & ( ~pd.isnull( merged_calendar['caregiver_gcal'] ) )
        & ( merged_calendar['caregiver'] != merged_calendar['caregiver_gcal'] )
    ]
if len( entries_to_update ) > 0:
    logger.info( 'Updating entries...' )
else:
    logger.info( 'No entries to be updated.' )
    
for ix, row in entries_to_update.iterrows():
    logger.info( f"* {row['date']}: {row['caregiver_gcal']} -> {row['caregiver']}" )
    service.events().delete( calendarId = gcal_calendar_id, eventId = row['event_id'] ).execute()
    event  = buildEvent( row )
    result = service.events().insert( calendarId = gcal_calendar_id, body = event ).execute()
    

2023-11-15 00:25:21,757 - RPESChickens - INFO - No entries to be deleted.
2023-11-15 00:25:21,761 - RPESChickens - INFO - Inserting entries...
2023-11-15 00:25:21,762 - RPESChickens - INFO - * 2023-11-23: Micah Rice
2023-11-15 00:25:22,314 - RPESChickens - INFO - * 2023-12-03: Brian Purchia
2023-11-15 00:25:22,966 - RPESChickens - INFO - * 2023-11-05: Kyle Woodward
2023-11-15 00:25:23,722 - RPESChickens - INFO - * 2023-11-04: Kyle Woodward
2023-11-15 00:25:24,495 - RPESChickens - INFO - * 2023-12-13: Micah Rice
2023-11-15 00:25:25,326 - RPESChickens - INFO - * 2023-12-10: Brian Purchia
2023-11-15 00:25:26,039 - RPESChickens - INFO - * 2023-11-30: Micah Rice
2023-11-15 00:25:26,808 - RPESChickens - INFO - * 2023-12-06: Micah Rice
2023-11-15 00:25:27,492 - RPESChickens - INFO - * 2023-11-11: Brian Purchia
2023-11-15 00:25:28,040 - RPESChickens - INFO - * 2023-12-17: Brian Purchia
2023-11-15 00:25:28,860 - RPESChickens - INFO - * 2023-12-24: Brian Purchia
2023-11-15 00:25:29,440 - RPESChi

In [10]:
#
# 8. Clean up our handles
#
try:
    stream_file.close()

except:
    pass